<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/infer_condition2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
This condition will not ask users what type of inference they want. It will generate inferences, and rate the uncommonness and sensitivity of the inferences.

In [3]:
import openai

openai.api_key = 'read API key'

# tokenizer = AutoTokenizer.from_pretrained("gpt2")  # GPT-3.5 & GPT-4 use GPT-2 tokenizer

def count_tokens(text):
    """Returns the number of tokens in a given text."""
    return len(tokenizer.encode(text))

def truncate_list(data_list, max_tokens):
    """Truncates a list to fit within a token budget."""
    truncated_list = []
    token_count = 0
    for item in data_list:
        item_tokens = count_tokens(str(item))  # Count tokens in each item
        if token_count + item_tokens <= max_tokens:
            truncated_list.append(item)
            token_count += item_tokens
        else:
            break
    return truncated_list

def run_prompt(prompt):
    response = openai.chat.completions.create(
        # model="gpt-3.5-turbo",  # Changed 'engine' to 'model'
        model="gpt-4o-mini",
        # model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
        # max_tokens=min(1000, 16385 - input_token_count)  # Ensure within token limit
        temperature=0.7,
        n=1, # Return only one response
        stop=None # optional stopping sequence
        # request_timeout = 60
        )
    json_response = response.choices[0].message.content.strip()

    return json_response  # Convert response to Python dictionary

In [4]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
import json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
powerset2023 = pd.read_csv('/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_by_year/allActivity_2023.csv')

In [3]:
def save_inference_json(inference_data, inference_no, columns_used):
    """Saves the inference data to a JSON file with an automated filename."""
    base_dir = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data"

    # Create a descriptive filename based on inference number and columns
    column_names = "_".join([col.split("_")[1] for col in columns_used])
    filename = f"inferences_{inference_no}_{len(columns_used)}cols_{column_names}.json"
    output_filename = f"{base_dir}/{filename}"

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(inference_data, f, indent=4, ensure_ascii=False)

    print(f"✅ JSON saved successfully as {output_filename}")

In [6]:
# Select columns with less than 600 non-NaN rows
misc_columns = powerset2023.columns[powerset2023.notna().sum() < 600]

# Combine values into a new column
powerset2023['takeout1_misc_MyActivity_Search Title'] = powerset2023[misc_columns].apply(
    lambda row: ', '.join(row.dropna().astype(str)),
    axis=1)

# Drop the identified misc_columns
powerset2023 = powerset2023.drop(columns=misc_columns)


# Drop duplicate values in the specified column, keeping the first occurrence
powerset2023 = powerset2023.drop_duplicates(subset=['takeout1_chrome_MyActivity_Search Title'], keep='first')

make prompt more dynamic --> ask for user input for inference and recommendation #

*prompt: recommend one product based on these three inferences*

make numeric instead of text --> parameterize the model --> multiply it out --> if you recommend one product based on the inferences; for example inference 1 recommendation 1, inference 1 recommendation 2 and so on

In [7]:
#Ask to select any four column
print("\nPlease select four datasets from the options (datasets should not be the same):")
for i, col in enumerate(powerset2023.columns, 1):
    print(f"{i}. {col}")

# Ask the user to select any 4 columns
selected_columns = []
df_selected = pd.DataFrame()
while len(selected_columns) < 4:
    try:
        col_num = int(input(f"\nSelect column {len(selected_columns) + 1} by number (1-{len(powerset2023.columns)}): "))
        if 1 <= col_num <= len(powerset2023.columns):
            col_name = powerset2023.columns[col_num - 1]
            if col_name not in selected_columns:
                selected_columns.append(col_name)
            else:
                print("You've already selected this column. Try another.")
        else:
            print("Invalid number. Please choose a number from the list.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Extract the selected columns
powerSubset2023 = powerset2023[selected_columns]

# Print the selected DataFrame
print("\nSelected DataFrame:")


Please select four datasets from the options (datasets should not be the same):
1. takeout1_imageSearch_MyActivity_Search Title
2. takeout1_chrome_MyActivity_Search Title
3. takeout1_maps_MyActivity_Search Title
4. takeout1_YT_search-history_Search Title
5. takeout1_YT_watch-history_Search Title
6. takeout1_misc_MyActivity_Search Title

Select column 1 by number (1-6): 1

Select column 2 by number (1-6): 2

Select column 3 by number (1-6): 4

Select column 4 by number (1-6): 6

Selected DataFrame:


In [ ]:
# # Optionally, save to a CSV file
save_option = input("\nWould you like to save this selection to a CSV file? (yes/no): ").strip().lower()
if save_option == "yes":
    df_selected.to_csv("/content/drive/MyDrive/Dissertation/Study 1/Inference_data/24APRIL2025/chrome_maps_YTwatch_misc.csv", index=False)
    # print("File saved as 'selected_input_data.csv'.")


Would you like to save this selection to a CSV file? (yes/no): no


In [8]:
#Ask to select how many inferences they wat

# Define the options
options = [3,6,9,12]

# Display the options
print("Please select a number from the options below by typing the corresponding number:\n")
for i, option in enumerate(options, 1):
    print(f"{i}. {option}")

# Get user input
user_input = input("\nEnter your selection (1-4): ")

# Validate input
if user_input.isdigit() and 1 <= int(user_input) <= 4:
    inference_no = options[int(user_input) - 1]
    print(f"\nYou selected: {inference_no}")
else:
    print("\nInvalid selection. Please enter a number between 1 and 4.")

# inference_input = int(inference_input)

Please select a number from the options below by typing the corresponding number:

1. 3
2. 6
3. 9
4. 12

Enter your selection (1-4): 1

You selected: 3


In [12]:
def clean_json_string(json_string):
  """Cleans a JSON string by removing unwanted characters and standardizing spacing."""

  # cleaned = re.sub(r"```json ", "", json_string)
  # Replace extra spaces and newlines with single spaces
  cleaned = re.sub(r"```json|```", " ", json_string)
  # Remove characters outside the basic multilingual plane
  cleaned = re.sub(r"[^\u0000-\uFFFF]", "", cleaned)
  # Try to parse the cleaned JSON; return original if invalid
  try:
    return json.loads(cleaned)
  except json.JSONDecodeError:
    print(f"Warning: Failed to parse JSON, returning original: {json_string[:50]}...")
    return cleaned

# Apply the function to each item in your list

In [9]:
def build_prompts_for_columns(powerSubset2023, col_names, index, inference_no, num_cols=4, rows=200):
    """
    Generate prompts for combinations of columns (2, 3, or 4).
    """
    inference_json_list = []

    # Outer loop keeps col_names[index] constant

    for index in range(len(col_names) - (num_cols - 1)):
        for next_index in range(index + 1, len(col_names) - (num_cols - 1)):
            # Select dynamic number of columns
            selected_cols = [col_names[index]] + col_names[next_index:next_index + (num_cols - 1)]

            columns_str = " AND ".join(selected_cols)

            # Fetch data for each column
            col_data_list = [powerSubset2023[:rows][col] for col in selected_cols]

            col_data_str = " AND ".join(
                f"{col_data.dropna().tolist()} in {col_name}"
                for col_data, col_name in zip(col_data_list, selected_cols)
            )

            prompt = f"""
            User wants to know what {col_data_str} tells about them.
            1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
            2. For each interest, rate uncommonness of the interest (scale 1-10, where 1 = most uncommon and 10 = most common).
            Uncommon refers to how rare or atypical an interest is among people who share similar demographic characteristics.
            3. For each interest, rate sensitivity of interest (scale 1–10, where 1 = least sensitive and 10 = most sensitive).
            Sensitivity refers to how likely an interest is to reveal personal, private, or stigmatized information about the user —
            especially information they may not want to be shared, inferred, or acted upon without their consent.
            It includes how much harm, discomfort, or unwanted exposure could result from the disclosure of that interest.
            4. Recommend **EXACTLY ONE** product for *EACH GROUP* identified in step 2.
            5. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{columns_str}**.

            **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**

            ```json
            {{
              "columns": "{columns_str}",
              "inferences": [
                {', '.join([json.dumps({
                    "inference": f"<output for inference {{i+1}}>",
                    "explanation_inference": f"<explanation for inference {{i+1}}>",
                    "commonness": f"<Inference {{i+1}} commonness score>",
                    "explanation_commonness": f"<Explanation for commonness score {{i+1}}>",
                    "sensitivity": f"<Inference {{i+1}} sensitivity score>",
                    "explanation_sensitivity": f"<Explanation for sensitivity score {{i+1}}>",
                    "recommendation": f"<recommendation for inference {{i+1}}>"
                    }) for i in range(inference_no)])}
                    ],
            "final_product_recommendation": {{
              "ONE recommendation based on ALL inferences": "<Product name and company>"
              }}
            }}
          ```
"""
        # ✅ Send prompt to GPT and collect response
        print(f"🔹 Sending prompt to GPT for columns: {columns_str}...")
        assistant_reply = run_prompt(prompt)
        print(assistant_reply)
        inference_json_list.append(assistant_reply)

    # ✅ Increment index only after inner loop finishes
    index += 1

    return inference_json_list


In [10]:
col_names = list(powerSubset2023.columns)

In [ ]:
# For 2-column combos
inference_c2_json = build_prompts_for_columns(powerSubset2023, col_names, index=0, inference_no=3, num_cols=2)

# For 4-column combos
inference_c4_json = build_prompts_for_columns(powerSubset2023, col_names, index=0, inference_no=3, num_cols=4)

# For 3-column combos
inference_c3_json = build_prompts_for_columns(powerSubset2023, col_names, index=0, inference_no=3, num_cols=3)

In [23]:
def write_json(output_filename, inference_json):
    if not inference_json:
        print("⚠️ No data to write. JSON file not created.")
        return None

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(inference_json, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON saved successfully as {output_filename}")
    return output_filename

In [24]:
output_filename_ = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/.json" # Define a suitable filename
write_json(output_filename_, [clean_json_string(item) for item in inference_c2_json])

✅ JSON saved successfully as /content/drive/MyDrive/Dissertation/Study 1/Inference_data/c2_inferences.json


'/content/drive/MyDrive/Dissertation/Study 1/Inference_data/c2_inferences.json'

Ask LLM while generating or retrospectively to rate each of these inferences as a level of uncommonness. Add a column with inference score of 1 to 10 for uncommonness. Higher score to uncommon inferences than some of the others.

Pick top N of the uncommon (unexpected), sensitive ones (including combinations) and use them

Robustness in repitition and generalizabiliy